In [1]:
# Initialisation

#q1 of ass1

import numpy as np
import math
import torch as t
import torch 

%matplotlib inline
# import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt


In [2]:

# part 1 and 2:
# dataset loading:
training_data = datasets.FashionMNIST(
    root="../../data/",
    train=True,
    download=True,
    transform=ToTensor() # Normalisation step
    
)

# note that ToTensor not just converts the image into a tensor but also normalises its intensity in range 0 to 1


test_data = datasets.FashionMNIST(
    root="../../data/",
    train=False,
    download=True,
    transform=ToTensor() # Normalisation step
)

# Create data loaders for training and testing

batchSize=64
train_loader = DataLoader(training_data, batch_size=batchSize, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batchSize, shuffle=False)

In [3]:
# part 3:

import torch.nn as nn

# builtin classifier:

# Define the model



model = nn.Sequential(

    # we need to flatten the input data from 2d tensor into 1d tensor of 784 values (28*28) since data is in form of images
    nn.Flatten(),  # Flatten the input data

    nn.Linear(in_features=784, out_features=10),  # Linear layer   
    # nn.Linear(in_features=784, out_features=batchSize),  # Linear layer   

    nn.BatchNorm1d(10),
    # this normalsises input tensors by scaling and shifting
    # makes mean 0 and variance 1

    # softmax function, or normalized exponential function converts a vector of K real numbers into a probability distribution of K possible outcomes
    # so basically coverts output labels into probablility distribution
    nn.Softmax(dim=1)  # Softmax activation
)


# custom made classifier
'''
class LinearClassifier(nn.Module):
    def __init__(self):
        super(LinearClassifier, self).__init__()

        # Define the linear layer
        # we used 28*28 input and 10 labels because we are working with MNIST database
        self.fc = nn.Linear(28 * 28, 10)  # 28*28 input features, 10 output classes
        
        # Define an activation function (e.g., softmax for classification)

        # softmax function, or normalized exponential function converts a vector of K real numbers into a probability distribution of K possible outcomes
        self.softmax = nn.Softmax(dim=1)

    #IMPORTANT the name 'forward' is not arbitrary; it's a convention in PyTorch, and this method is automatically called when we pass data through our model.
    def forward(self, x):

        # we need to flatten the input data from 2d tensor into 1d tensor of 784 values (28*28) since data is in form of images
        x = x.view(x.size(0), -1)  # Flatten the input tensor

        x = self.fc(x)  # Apply the linear transformation
        x = self.softmax(x) # Apply the activation function
        return x
'''
print('model created')

model created


In [4]:
print("Model structure before training: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure before training:  Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Softmax(dim=1)
) 


Layer: 1.weight | Size: torch.Size([10, 784]) | Values : tensor([[-0.0146, -0.0063, -0.0253,  ..., -0.0298,  0.0181,  0.0133],
        [ 0.0036,  0.0027,  0.0063,  ...,  0.0213, -0.0095,  0.0348]],
       grad_fn=<SliceBackward0>) 

Layer: 1.bias | Size: torch.Size([10]) | Values : tensor([-0.0334, -0.0048], grad_fn=<SliceBackward0>) 

Layer: 2.weight | Size: torch.Size([10]) | Values : tensor([1., 1.], grad_fn=<SliceBackward0>) 

Layer: 2.bias | Size: torch.Size([10]) | Values : tensor([0., 0.], grad_fn=<SliceBackward0>) 



In [5]:
# part 4:

# Define the loss function
criterion = nn.CrossEntropyLoss()

import torch.optim as optim
# Define the optimizer (e.g., SGD)

learning_rate=0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

'''
about SGD: Stochastic Gradient Descent:
basically it performs seqeuntial updation of paramters after each epoch, using formula :
parameter = parameter - learning_rate * gradient

'''

print('part 4 done')



part 4 done


In [9]:
# part 5
# Define the number of training epochs
num_epochs = 10

import time
t1=time.time()
# Training loop
for epoch in range(num_epochs):
    # Set the model in training mode
    model.train()
    
    running_loss = 0.0
    
    # Iterate over the training dataset
    for inputs, labels in train_loader:  # Replace train_loader with your data loader
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backpropagation
        loss.backward()
        
        # Update the model's parameters
        optimizer.step()
        
        # Track the running loss
        running_loss += loss.item()
    
    # Calculate and print the average loss for this epoch
    average_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {average_loss:.4f}")

print("Training complete")
t2=time.time()
training_time=t2-t1
print('time taken to train : ',training_time)

Epoch [1/10] - Loss: 1.6797
Epoch [2/10] - Loss: 1.6759
Epoch [3/10] - Loss: 1.6723
Epoch [4/10] - Loss: 1.6697
Epoch [5/10] - Loss: 1.6657
Epoch [6/10] - Loss: 1.6632
Epoch [7/10] - Loss: 1.6614
Epoch [8/10] - Loss: 1.6595
Epoch [9/10] - Loss: 1.6585
Epoch [10/10] - Loss: 1.6562
Training complete
time taken to train :  110.4238793849945


In [10]:
print("Model structure after training: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure after training:  Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Softmax(dim=1)
) 


Layer: 1.weight | Size: torch.Size([10, 784]) | Values : tensor([[-0.0146, -0.0061, -0.0245,  ..., -0.0702, -0.0068,  0.0113],
        [ 0.0036,  0.0027,  0.0063,  ...,  0.0051, -0.0106,  0.0342]],
       grad_fn=<SliceBackward0>) 

Layer: 1.bias | Size: torch.Size([10]) | Values : tensor([-0.0334, -0.0048], grad_fn=<SliceBackward0>) 

Layer: 2.weight | Size: torch.Size([10]) | Values : tensor([2.7418, 2.7357], grad_fn=<SliceBackward0>) 

Layer: 2.bias | Size: torch.Size([10]) | Values : tensor([0.0559, 0.0244], grad_fn=<SliceBackward0>) 



In [11]:
# part 6 :
# testing 

# Set the model to evaluation mode
model.eval()

correct_predictions = 0
total_predictions = 0

# Iterate over the testing dataset

k=0
for inputs, labels in test_loader:  # Use your test data loader
    k+=1
    # Forward pass to obtain predictions
    outputs = model(inputs)
    
    # Get the predicted class for each example in the batch
    _, predicted = torch.max(outputs, dim=1)
    
    # Count correct predictions in this batch
    batch_correct_predictions = (predicted == labels).sum().item()
    correct_predictions += batch_correct_predictions
    
    # Count total predictions in this batch

    batch_total_predictions = labels.size(0)
    total_predictions += batch_total_predictions 

    # Print the running accuracy for this batch
    # Update running accuracy
    batch_accuracy=batch_correct_predictions/batch_total_predictions * 100.0
    print(f"Accuracy on this batch {batch_accuracy:.2f}%")

    accuracy = (correct_predictions / total_predictions) * 100.0
    print(f"Accuracy till now:  testing dataset: {accuracy:.2f}%")

print()
print("number of tests:",k)

# Calculate the accuracy
accuracy = (correct_predictions / total_predictions) * 100.0
print(f"Accuracy on the testing dataset: {accuracy:.2f}%")
print('time taken to train : ',training_time)


Accuracy on this batch 82.81%
Accuracy till now:  testing dataset: 82.81%
Accuracy on this batch 87.50%
Accuracy till now:  testing dataset: 85.16%
Accuracy on this batch 87.50%
Accuracy till now:  testing dataset: 85.94%
Accuracy on this batch 82.81%
Accuracy till now:  testing dataset: 85.16%
Accuracy on this batch 87.50%
Accuracy till now:  testing dataset: 85.62%
Accuracy on this batch 89.06%
Accuracy till now:  testing dataset: 86.20%
Accuracy on this batch 90.62%
Accuracy till now:  testing dataset: 86.83%
Accuracy on this batch 85.94%
Accuracy till now:  testing dataset: 86.72%
Accuracy on this batch 82.81%
Accuracy till now:  testing dataset: 86.28%
Accuracy on this batch 78.12%
Accuracy till now:  testing dataset: 85.47%
Accuracy on this batch 78.12%
Accuracy till now:  testing dataset: 84.80%
Accuracy on this batch 85.94%
Accuracy till now:  testing dataset: 84.90%
Accuracy on this batch 89.06%
Accuracy till now:  testing dataset: 85.22%
Accuracy on this batch 87.50%
Accuracy